In [ ]:
#import sys
#print(sys.version)

In [ ]:
%pip install ultralytics

In [ ]:
import ultralytics
ultralytics.__version__

In [ ]:
# Not training the model, just using pretrained model trained on COCO dataset. 
# Aquest dataset té som vehicles a part d'altres objectes.
#Fer servir aquest model per la detecció i després track i count
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*

model=YOLO('yolov5n.pt')

In [ ]:
#list of COCO dataset classes
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [ ]:
#creant una instancia de tracker que ve de from tracker import*
tracker=Tracker()
count=0

In [ ]:
#llegir el video
cap=cv2.VideoCapture('C:/Users/macia/OneDrive/Escriptori/4t GED/PSIV/Repte2-proves/output7.mp4')

In [ ]:
#VARIABLES PER GUARDAR COTXES UP I DOWN
down={}
up={}

counter_down=[]
counter_up=[]

In [ ]:
import time

# Inicialitzar el comptador de FPS
fps = 0
start_time = time.time()

# Bucle infinit per processar el vídeo frame a frame
while True:
    # Llegir un nou frame de la captura de vídeo
    ret, frame = cap.read()
    if not ret:  # Si no hi ha més frames, surt del bucle
        break
    count += 1

    # RESOLUCIÓ IMATGE ORIGINAL: 960X540
    # REDUIR RESOLUCIÓ a 270x480
    frame = cv2.resize(frame, (270, 480))

    # Saltar 1 de cada 2 frames per processar menys imatges
    if count % 2 != 0:
        fps += 1
        end_time = time.time()
        elapsed_time = end_time - start_time
        if elapsed_time > 0:
            # Calcular FPS actuals
            current_fps = fps / elapsed_time
            # Mostrar els FPS al frame
            cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            # Dibuixar la línia única al frame per detectar el moviment
            cv2.line(frame, (0, 400), (800, 400), (0, 0, 255), 3)  # línia vermella (única)
        # Mostrar el frame
        cv2.imshow("frames", frame)
        if cv2.waitKey(1) & 0xFF == 27:  # Si es prem ESC, sortir del bucle
            break
        continue

    # Predicció amb YOLO
    results = model.predict(frame)

    # Obtenir les dades de la bounding box i la confiança de la predicció
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    # Filtrar només les bounding boxes de cotxes
    car_bboxes = []
    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]  # Obtenir la classe de l'objecte detectat
        if 'car' in c:  # Si l'objecte és un cotxe, guardar la seva bounding box
            car_bboxes.append([x1, y1, x2, y2])

    # Proporcionar les coordenades de les bounding boxes al tracker
    bbox_id = tracker.update(car_bboxes)

    # Dibuixar les bounding boxes i els IDs al frame
    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int((x3 + x4) // 2)  # Coordenada central de la bounding box (x)
        cy = int((y3 + y4) // 2)  # Coordenada central de la bounding box (y)

        # Actualitzar comptadors de cotxes que baixen o pugen
        line_y = 400  # Coordenada y de la línia única
        offset = 7  # Margen d'error per la detecció de passos per la línia

        # Si el cotxe estava a la part superior i després passa a la part inferior, comptar com a baixada
        if cy < line_y - offset:  # Cotxe a la part superior de la línia
            down[id] = cy  # Guardar la y del cotxe quan està a la part superior
        if id in down:
            if cy > line_y + offset:  # Cotxe passa a la part inferior
                # Dibuixar un punt vermell al cotxe que baixa
                cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punt vermell
                # Mostrar l'ID del cotxe al frame
                cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                counter_down.append(id)  # Afegir l'ID a la llista de cotxes que baixen

        # Si el cotxe estava a la part inferior i després passa a la part superior, comptar com a pujada
        if cy > line_y + offset:  # Cotxe a la part inferior de la línia
            up[id] = cy  # Guardar la y del cotxe quan està a la part inferior
        if id in up:
            if cy < line_y - offset:  # Cotxe passa a la part superior
                # Dibuixar un punt vermell al cotxe que puja
                cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # punt vermell
                # Mostrar l'ID del cotxe al frame
                cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                counter_up.append(id)  # Afegir l'ID a la llista de cotxes que pugen

    # Dibuixar la línia vermella al frame
    cv2.line(frame, (0, 400), (800, 400), (0, 0, 255), 3)  # línia vermella (única)

    # Mostrar el nombre de cotxes que han baixat i pujat
    downwards = len(set(counter_down))  # Comptar cotxes únics que han baixat
    upwards = len(set(counter_up))  # Comptar cotxes únics que han pujat

    # Imprimir els resultats
    print(f"Baixada: {downwards}")
    print(f"Pujada: {upwards}")

    # Càlcul de FPS (frames per segon)
    fps += 1
    end_time = time.time()
    elapsed_time = end_time - start_time
    if elapsed_time > 0:
        # Calcular i mostrar els FPS al frame
        current_fps = fps / elapsed_time
        cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Mostrar el frame actualitzat
    cv2.imshow("frames", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # Sortir si es prem la tecla ESC
        break

# Alliberar la captura de vídeo i destruir totes les finestres
cap.release()
cv2.destroyAllWindows()
